# Advent of code 2020: Day 7

## Import

In [1]:
import math as M
import collections as C
import bahnslib as B
import itertools as I


day = 7
data = B.get_aoc_input(day, f"../inputs/{str(day).zfill(2)}.txt", format="lines")

## Common
Create a dictionary linking bags and their contents, and another for the opposite

In [2]:
contents_of = {}
containing = {}
all_colours = set()
sub_colours = set()
grandparents = set()
grandchildren = set()

for line in data:
    parent, children = line[:-1].split(" bags contain ")
    all_colours.add(parent)
    children = children.replace("bags", "").replace("bag", "").split(", ")
    contents_of[parent] = {}
    for child in children:
        child = child.split()
        colour = " ".join(child[1:])
        if child != ["no", "other"]:
            sub_colours.add(colour)
            count = int(child[0])
            contents_of[parent][colour] = count 
            if colour in containing:
                containing[colour].add(parent)
            else:
                containing[colour] = {parent}
        else:
            grandchildren.add(parent)

grandparents = all_colours - sub_colours
for grandparent in grandparents:
    containing[grandparent] = []

## Part 1

In [3]:
parents = set()
def get_parents(colour):
    for parent in containing[colour]:
        parents.add(parent)
        get_parents(parent)
get_parents("shiny gold")

print(len(parents))


259


## Part 2

In [4]:
def get_children(colour, weight):
    t = 1
    for bag in contents_of[colour]:
        t += get_children(bag, contents_of[colour][bag])
    return t * weight

print(get_children("shiny gold", 1) - 1)


45018


## And now for some unnecessary bits and pieces

### Bad graphing

In [5]:
from graphviz import Digraph, Graph
import pyyed

graphed_children = set()
graphed_parents = set()


node_height = "0"
node_width = "0"
node_colour = "#000000"
label_size = "0"
yed = pyyed.Graph()
yed.add_node("shiny gold", shape_fill="#F9CB68", height=node_height, width=node_width)


def graph_children(colour): 
    if not(colour in yed.nodes):
            yed.add_node(colour, shape_fill=node_colour, height=node_height, width=node_width, font_size=label_size)
    for bag in contents_of[colour]:
        if not(bag in yed.nodes):
            yed.add_node(bag, shape_fill=node_colour, height=node_height, width=node_width, font_size=label_size)
        yed.add_edge(colour, bag, arrowhead="none")
        
        if bag not in graphed_children:
            graph_children(bag)
    graphed_children.add(colour)

def graph_parents(colour):
    if not(colour in yed.nodes):
        yed.add_node(colour, shape_fill=node_colour, height=node_height, width=node_width, font_size=label_size)
        
    for parent in containing[colour]:
        if not(parent in yed.nodes):
            yed.add_node(parent, shape_fill=node_colour, height=node_height, width=node_width, font_size=label_size)
        yed.add_edge(parent, colour, arrowhead="none")
        
        if parent not in graphed_parents:
            graph_parents(parent)
    graphed_parents.add(colour)

for grandparent in grandparents:
    graph_children(grandparent)
    
# graph_children("shiny gold")
# graph_parents("shiny gold")

yed.write_graph("full.graphml", pretty_print=True)